In [1]:
from mp_actors import move_to_child_process
import asyncio


class Service:
    async def load_unsloth(self) -> None:
        import unsloth

    async def greet(self, name: str, sleep: float) -> str:
        await asyncio.sleep(sleep)
        return f"Hello, {name}!"

    def raise_error(self) -> None:
        raise ValueError("This is a test error")


service = Service()
service = move_to_child_process(service)

In [2]:
service.raise_error()

ValueError: This is a test error

In [ ]:
import sys
import types


def get_child_traceback(
    traceback: types.TracebackType,
) -> tuple[types.TracebackType, bool]:
    if not traceback.tb_next:
        return traceback, False
    next_traceback, mp_actor_code = get_child_traceback(traceback.tb_next)
    if mp_actor_code:
        return next_traceback, True
    elif "/mp_actors/" in traceback.tb_frame.f_code.co_filename:
        return next_traceback, True
    return traceback, False


def get_parent_traceback(traceback: types.TracebackType) -> types.TracebackType:
    while traceback.tb_next:
        filename = traceback.tb_next.tb_frame.f_code.co_filename
        print(filename)
        if "/mp_actors/" in filename:
            return traceback
        traceback = traceback.tb_next
    return traceback


def get_traceback(traceback: types.TracebackType) -> types.TracebackType:
    parent_traceback = get_parent_traceback(traceback)
    child_traceback, _ = get_child_traceback(parent_traceback)
    return types.TracebackType(
        child_traceback,
        parent_traceback.tb_frame,
        parent_traceback.tb_lasti,
        parent_traceback.tb_lineno,
    )


try:
    service.raise_error()
except ValueError as e:
    traceback = sys.exc_info()[2]
    assert traceback
    display(traceback.tb_frame.f_code.co_filename)

    raise e.with_traceback(get_traceback(traceback))

'/tmp/ipykernel_153850/1131703420.py'

/home/gcpuser/sky_workdir/src/mp_actors/move.py


ValueError: This is a test error

In [2]:
await service.load_unsloth()
await service.greet("World", 1.0)

'Hello, World!'